In [1]:
# Events pro Tag und Raum

import json
from collections import defaultdict

# Datei laden
with open("data/2025/pretalx.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Struktur vorbereiten
event_count = defaultdict(lambda: defaultdict(int))

# Zugriff auf alle Tage im Schedule
days = data["schedule"]["conference"]["days"]

# Pro Tag und Raum die Anzahl der Events zählen
for day in days:
    date = day["date"]
    for room_name, events in day["rooms"].items():
        event_count[date][room_name] += len(events)

# Übersichtliche Ausgabe
for date, rooms in event_count.items():
    print(f"\n📅 {date}")
    for room, count in sorted(rooms.items()):
        print(f"  🏛 {room}: {count} Events")



📅 2025-03-26
  🏛 Bof1 (S8): 1 Events
  🏛 Bof2 (S9): 2 Events
  🏛 Bof3/Expert:innen: 1 Events
  🏛 HS1 (Aula): 10 Events
  🏛 HS2 (S10): 9 Events
  🏛 HS3 (S1): 8 Events
  🏛 HS4 (S2): 11 Events
  🏛 WS1 (106): 2 Events
  🏛 WS2 (107): 2 Events
  🏛 WS3 (108): 2 Events

📅 2025-03-27
  🏛 Bof1 (S8): 5 Events
  🏛 Bof2 (S9): 3 Events
  🏛 Bof3/Expert:innen: 4 Events
  🏛 HS1 (Aula): 12 Events
  🏛 HS2 (S10): 12 Events
  🏛 HS3 (S1): 13 Events
  🏛 HS4 (S2): 13 Events
  🏛 Poster (Zelt): 18 Events
  🏛 WS1 (106): 4 Events
  🏛 WS2 (107): 4 Events
  🏛 WS3 (108): 3 Events

📅 2025-03-28
  🏛 Bof1 (S8): 2 Events
  🏛 Bof2 (S9): 2 Events
  🏛 Bof3/Expert:innen: 1 Events
  🏛 HS1 (Aula): 11 Events
  🏛 HS2 (S10): 15 Events
  🏛 HS3 (S1): 6 Events
  🏛 HS4 (S2): 7 Events
  🏛 WS1 (106): 1 Events
  🏛 WS2 (107): 2 Events
  🏛 WS3 (108): 1 Events

📅 2025-03-29
  🏛 Bof1 (S8): 6 Events
  🏛 Bof2 (S9): 5 Events
  🏛 Bof3/Expert:innen: 4 Events
  🏛 HS1 (Aula): 1 Events


In [3]:
import json
from collections import defaultdict

# JSON einlesen
with open("data/2025/pretalx.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Dict: Raum → [total_events, recorded_events]
record_counts = defaultdict(lambda: [0, 0])

# Durch alle Events iterieren
for day in data["schedule"]["conference"]["days"]:
    for room, events in day["rooms"].items():
        for event in events:
            record_counts[room][0] += 1
            if not event.get("do_not_record", False):
                record_counts[room][1] += 1

# Sortieren nach Anzahl der Aufzeichnungen (absteigend)
sorted_rooms = sorted(record_counts.items(), key=lambda x: x[1][1], reverse=True)

# Ausgabe + Zähler
total_events = 0
total_recorded = 0

print("🎥 Aufzeichnungen pro Raum (Recorded / Total):")
for room, (total, recorded) in sorted_rooms:
    print(f"  {room:<20} {recorded:>2} / {total}")
    total_events += total
    total_recorded += recorded

# Summen anzeigen
print("\n📊 Gesamtsumme:")
print(f"  🎞 Aufgezeichnete Events: {total_recorded}")
print(f"  📋 Gesamte Events:        {total_events}")


🎥 Aufzeichnungen pro Raum (Recorded / Total):
  HS2 (S10)            34 / 36
  HS1 (Aula)           31 / 34
  HS4 (S2)             31 / 31
  HS3 (S1)             26 / 27
  WS2 (107)             8 / 8
  Poster (Zelt)         7 / 18
  Bof1 (S8)             6 / 14
  Bof3/Expert:innen     6 / 10
  WS1 (106)             6 / 7
  Bof2 (S9)             5 / 12
  WS3 (108)             5 / 6

📊 Gesamtsumme:
  🎞 Aufgezeichnete Events: 165
  📋 Gesamte Events:        203


In [4]:
import json
import os

def export_schedule_segment(input_path, output_dir, day_index, room_name):
    # Datei laden
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Konferenztage extrahieren
    days = data["schedule"]["conference"]["days"]

    # Sicherheitscheck
    if not (0 <= day_index - 1 < len(days)):
        raise ValueError(f"Ungültiger Tag: {day_index}")

    day = days[day_index - 1]
    events = day["rooms"].get(room_name)

    if not events:
        raise ValueError(f"Keine Events für Raum: {room_name}")

    # Nur relevante Daten extrahieren
    segment = {
        "date": day["date"],
        "room": room_name,
        "events": events
    }

    # Slug erzeugen
    room_slug = room_name.lower().replace(" ", "").replace("(", "").replace(")", "").replace("/", "-")
    filename = f"schedule-part-{day_index:02d}-{room_slug}.json"
    output_path = os.path.join(output_dir, filename)

    # Export
    os.makedirs(output_dir, exist_ok=True)
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(segment, f, indent=2, ensure_ascii=False)

    print(f"✅ Exportiert


SyntaxError: unterminated string literal (detected at line 39) (2810990289.py, line 39)

In [9]:
# Methode um HS1-HS4 und D1 und D4 zu splitten


import json
import os

def split_main_rooms_by_day(input_path, output_dir):
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    days = data["schedule"]["conference"]["days"]
    main_rooms = ["HS1 (Aula)", "HS2 (S10)", "HS3 (S1)", "HS4 (S2)"]

    os.makedirs(output_dir, exist_ok=True)

    for i, day in enumerate(days, start=1):
        date = day["date"]
        for room in main_rooms:
            events = day["rooms"].get(room)
            if not events:
                continue

            # Kürzel extrahieren (z. B. 'HS1' aus 'HS1 (Aula)')
            short_room = room.split(" ")[0].lower()
            filename = f"schedule-{short_room}-{i:02d}.json"
            output_path = os.path.join(output_dir, filename)

            segment = {
                "day": i,
                "date": date,
                "room": room,
                "events": events
            }

            with open(output_path, "w", encoding="utf-8") as f_out:
                json.dump(segment, f_out, indent=2, ensure_ascii=False)

            print(f"✅ Exportiert: {filename} ({len(events)} Events)")


In [8]:
# Aufruf von split_main_rooms

split_main_rooms_by_day(
    input_path="data/2025/pretalx.json",
    output_dir="data/2025/parts"
)


✅ Exportiert: schedule-hs1-01.json (10 Events)
✅ Exportiert: schedule-hs2-01.json (9 Events)
✅ Exportiert: schedule-hs3-01.json (8 Events)
✅ Exportiert: schedule-hs4-01.json (11 Events)
✅ Exportiert: schedule-hs1-02.json (12 Events)
✅ Exportiert: schedule-hs2-02.json (12 Events)
✅ Exportiert: schedule-hs3-02.json (13 Events)
✅ Exportiert: schedule-hs4-02.json (13 Events)
✅ Exportiert: schedule-hs1-03.json (11 Events)
✅ Exportiert: schedule-hs2-03.json (15 Events)
✅ Exportiert: schedule-hs3-03.json (6 Events)
✅ Exportiert: schedule-hs4-03.json (7 Events)
✅ Exportiert: schedule-hs1-04.json (1 Events)


In [12]:
# Methoden-Definition von export_small_rooms

import json
import os

def export_small_rooms(input_path, output_dir):
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    days = data["schedule"]["conference"]["days"]

    # Mapping: Originalname → Slug (angepasst: "bof3" statt "bof3-expertinnen")
    room_map = {
        "WS2 (107)": "ws2",
        "Poster (Zelt)": "poster",
        "Bof1 (S8)": "bof1",
        "Bof3/Expert:innen": "bof3",   # <- hier geändert
        "WS1 (106)": "ws1",
        "Bof2 (S9)": "bof2",
        "WS3 (108)": "ws3"
    }

    # Vorbereitung
    room_events = {slug: [] for slug in room_map.values()}
    os.makedirs(output_dir, exist_ok=True)

    # Sammeln
    for day in days:
        for room_full, events in day["rooms"].items():
            if room_full in room_map:
                slug = room_map[room_full]
                room_events[slug].extend(events)

    # Schreiben
    for slug, events in room_events.items():
        output_path = os.path.join(output_dir, f"schedule-{slug}.json")
        segment = {
            "room": slug,
            "events": events
        }

        with open(output_path, "w", encoding="utf-8") as f_out:
            json.dump(segment, f_out, indent=2, ensure_ascii=False)

        print(f"✅ Exportiert: schedule-{slug}.json ({len(events)} Events)")


In [13]:
# Aufruf von export_small_rooms

export_small_rooms(
    input_path="data/2025/pretalx.json",
    output_dir="data/2025/parts"
)


✅ Exportiert: schedule-ws2.json (8 Events)
✅ Exportiert: schedule-poster.json (18 Events)
✅ Exportiert: schedule-bof1.json (14 Events)
✅ Exportiert: schedule-bof3.json (10 Events)
✅ Exportiert: schedule-ws1.json (7 Events)
✅ Exportiert: schedule-bof2.json (12 Events)
✅ Exportiert: schedule-ws3.json (6 Events)
